<a href="https://colab.research.google.com/github/rodrigo0345/Aprof2/blob/main/Task2_YOLO_RESNET50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.optim import SGD, Adam
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
import torchvision.transforms as transforms
from torchvision.transforms import Compose
from torchvision.transforms import ToTensor
from torchvision.transforms import Normalize
from torchvision.transforms import Compose, Lambda, Normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, mean_absolute_error
import torch
import torch.nn.functional as F
import torchvision
from torchvision.transforms import Normalize
import torch.optim as optim
import seaborn as sns

# openpyxl é preciso ser instalado para TPUs
!pip install decord openpyxl ultralytics
import decord
from decord import VideoReader
import os
# from torchinfo import summary
# from livelossplot import PlotLosses
from ultralytics import YOLO


np.random.seed(0)
torch.manual_seed(0)
import random
random.seed(0)

In [ ]:
from ultralytics import YOLO
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

try:
    print("Attempting to load YOLO model...")
    model = YOLO('yolov8n.pt') # Ou o modelo que você usa
    model.to(device)
    model.eval() # Coloque em modo de avaliação
    print("YOLO model loaded and set to eval mode successfully.")

    # Crie um tensor de entrada dummy
    dummy_input = torch.rand(1, 3, 224, 224).to(device) # Ajuste o tamanho se necessário
    print(f"Dummy input created with shape: {dummy_input.shape} on device: {dummy_input.device}")

    print("Attempting YOLO prediction...")
    with torch.no_grad(): # Importante para inferência
        results = model(dummy_input, verbose=False) # verbose=False para menos output
    print("YOLO prediction successful.")

    if results and len(results) > 0 and hasattr(results[0], 'boxes') and results[0].boxes is not None:
        print(f"Number of boxes detected in the first result: {len(results[0].boxes)}")
    else:
        print("No boxes detected or results format is unexpected.")

except Exception as e:
    print(f"AN ERROR OCCURRED DURING ISOLATED YOLO TEST: {e}")
    import traceback
    traceback.print_exc()

PyTorch version: 2.6.0+cu124
CUDA available: False
Using device: cpu
Attempting to load YOLO model...
YOLO model loaded and set to eval mode successfully.
Dummy input created with shape: torch.Size([1, 3, 224, 224]) on device: cpu
Attempting YOLO prediction...
YOLO prediction successful.
Number of boxes detected in the first result: 0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "drive/MyDrive/suturas-filtered/"
labels_path = "drive/MyDrive/OSATS.xlsx"

info_df = pd.read_excel(labels_path)

# Get list of video files in the directory
video_files = [f.replace("output_prewitt_", "").replace(".mp4", "") for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith(('.mp4', '.avi', '.mov'))]
video_column = 'VIDEO'

info_df = info_df[info_df[video_column].isin(video_files)]
print(f"Number of videos after filtering: {len(info_df)}")
info_df.head()

Number of videos after filtering: 90


,STUDENT,GROUP,TIME,SUTURES,INVESTIGATOR,VIDEO,OSATS_RESPECT,OSATS_MOTION,OSATS_INSTRUMENT,OSATS_SUTURE,OSATS_FLOW,OSATS_KNOWLEDGE,OSATS_PERFORMANCE,OSATS_FINAL_QUALITY,GLOBA_RATING_SCORE
21,ALN135,E-LEARNING,POST,3.5,A,A61V,4,2,3,3,2,3,3,4,24
22,ALN135,E-LEARNING,POST,3.5,B,A61V,4,3,4,3,3,4,3,4,28
23,ALN135,E-LEARNING,POST,3.5,C,A61V,4,3,2,3,3,3,3,3,24
42,BOG917,HMD-BASED,PRE,2.5,A,C16G,3,3,3,2,2,2,2,1,18
43,BOG917,HMD-BASED,PRE,2.5,B,C16G,2,2,3,2,3,3,3,3,21


In [ ]:
OSATS_COLUMNS_INDIVIDUAL = [
    'OSATS_RESPECT', 'OSATS_MOTION', 'OSATS_INSTRUMENT',
    'OSATS_SUTURE', 'OSATS_FLOW', 'OSATS_KNOWLEDGE',
    'OSATS_PERFORMANCE', 'OSATS_FINAL_QUALITY'
]

def get_vector_of_ratings(video: str, source_df: pd.DataFrame):
    video_data = info_df[source_df['VIDEO'] == video]
    return video_data[OSATS_COLUMNS_INDIVIDUAL].iloc[0].values.tolist()

print(get_vector_of_ratings("A31H",info_df))


def update_info_df_with_binned_grs(df: pd.DataFrame):
    updated_df = df.copy()

    binned_scores_series = updated_df['VIDEO'].apply(
        lambda video_id: get_vector_of_ratings(video_id, updated_df)
    )

    scores_df_temp = pd.DataFrame(
        binned_scores_series.tolist(),
        columns=OSATS_COLUMNS_INDIVIDUAL,
        index=updated_df.index
    )

    for col_name in OSATS_COLUMNS_INDIVIDUAL:
        updated_df[col_name] = scores_df_temp[col_name]

    osats_score_vector_list = []
    videos_with_issues_for_vector = 0
    for index, row in updated_df.iterrows():
        current_vector = []
        valid_vector = True
        for col_name in OSATS_COLUMNS_INDIVIDUAL:
            score = row[col_name]
            if pd.isna(score):
                valid_vector = False
                break
            try:
                current_vector.append(float(score))
            except (ValueError, TypeError):
                valid_vector = False
                break

        if valid_vector and len(current_vector) == len(OSATS_COLUMNS_INDIVIDUAL):
            osats_score_vector_list.append(current_vector)
        else:
            osats_score_vector_list.append(None)
            videos_with_issues_for_vector += 1

    updated_df['OSATS_SCORE_VECTOR'] = osats_score_vector_list

    if updated_df['OSATS_SCORE_VECTOR'].isnull().any():
         num_null_vectors = updated_df['OSATS_SCORE_VECTOR'].isnull().sum()

    return updated_df

info_df = update_info_df_with_binned_grs(info_df)

[1, 2, 1, 1, 1, 2, 2, 1]


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
         for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

print(device)

cpu


In [ ]:
# mean = 0.0
# std = 0.0
# total_images_count = 0
#
# for images, _ in loader:
#     batch_samples = images.size(0) # batch size
#     images = images.view(batch_samples, images.size(1), -1) # Aplaina altura e largura
#     mean += images.mean([0, 2]).sum() # Média sobre canais e pixels, soma sobre batch
#     std += images.std([0, 2]).sum()   # Desvio sobre canais e pixels, soma sobre batch
#     total_images_count += batch_samples
#
# mean /= total_images_count
# std /= total_images_count
#
# print(f"Calculated mean: {mean.item()}") # .item() para obter o valor Python
# print(f"Calculated std: {std.item()}")
#
# # E então use estes valores:
# # train_transform = Compose([
# #     ToTensor(),
# #     Normalize(mean=(mean.item(),), std=(std.item(),))
# # ])

train_transform = Compose([
    ToTensor()
])

test_transform = Compose([
    ToTensor()
])

In [ ]:
batch_size = 1
segments = 16

In [ ]:
class VideoDataset(Dataset):
    def __init__(self, df_dataset, cfg, dataset_path, segments, segment_frame_pos=0) -> None:
        super().__init__()
        self.df_dataset = df_dataset
        self.dataset_path = dataset_path
        self.cfg = cfg
        self.label = [
            'OSATS_RESPECT', 'OSATS_MOTION', 'OSATS_INSTRUMENT',
            'OSATS_SUTURE', 'OSATS_FLOW', 'OSATS_KNOWLEDGE',
            'OSATS_PERFORMANCE', 'OSATS_FINAL_QUALITY'
        ]
        self._segments = segments
        self._segment_frame_pos = segment_frame_pos

    def __len__(self):
        return len(self.df_dataset)

    def __getitem__(self, idx):
        video_path = os.path.join(self.dataset_path, "output_prewitt_" + self.df_dataset.iloc[idx]["VIDEO"] + ".mp4")
        vr = decord.VideoReader(video_path)
        total_frames = len(vr)

        num_frames = self.cfg["num_frames"]
        if total_frames >= num_frames:
            indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
        else:
            indices = list(range(total_frames)) + [total_frames - 1] * (num_frames - total_frames)

        if self._segments is not None:
          # Divide indices into subsets
          subsets = np.array_split(indices, self._segments)
          # Retrieve the first element of each subset
          indices = [subset[self._segment_frame_pos] for subset in subsets]

        frames = vr.get_batch(indices).asnumpy()  # [T, H, W, C], uint8

        # Convert to tensor and normalize to [0, 1]
        frames = torch.from_numpy(frames).float() / 255.0

        # Permute to [T, C, H, W]
        frames = frames.permute(0, 3, 1, 2)

        # Normalize with mean and std
        mean = [0.45, 0.45, 0.45]
        std = [0.225, 0.225, 0.225]
        normalize = Normalize(mean, std)
        frames = normalize(frames)

        # Short side scale
        side_size = 256
        H, W = vr[0].shape[0], vr[0].shape[1]
        if H < W:
            new_H = side_size
            new_W = int(side_size * W / H)
        else:
            new_W = side_size
            new_H = int(side_size * H / W)

        # Resize using interpolate
        frames = F.interpolate(frames, size=(new_H, new_W), mode='bilinear', align_corners=False)

        # Center crop to 256x256
        crop_size = 256
        start_h = (new_H - crop_size) // 2
        start_w = (new_W - crop_size) // 2
        frames = frames[:, :, start_h:start_h + crop_size, start_w:start_w + crop_size]

        frames = frames.permute(1, 0, 2, 3)

        if isinstance(self.label, str):
            label = self.df_dataset.iloc[idx][self.label]
            label = torch.tensor(label, dtype=torch.long)
        else:
            label = [self.df_dataset.iloc[idx][col] for col in self.label]
            label = torch.tensor(label, dtype=torch.long)

        return frames, label

# task = 1 | 2
def prepare_data_loaders(path_train, task=1, segments=None):
    train_df, temp_df = train_test_split(info_df, test_size=0.3, random_state=42)
    val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

    cfg = {
        "num_frames": 64 # isto não me convence muito
    }
    train_dataset = VideoDataset(train_df, cfg, path_train, segments)
    val_dataset = VideoDataset(val_df, cfg, path_train, segments)
    test_dataset = VideoDataset(test_df, cfg, path_train, segments)

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
    )

    return train_loader, val_loader, test_loader

train_dl, val_dl, test_dl = prepare_data_loaders(path, task = 2, segments=segments)

In [ ]:
# Cell 10: YOLO + ResNet Model Definition
# NUM_OSATS_CATEGORIES = 8
# NUM_SCORES_PER_CATEGORY = 5
# RESNET_INPUT_SIZE = (224, 224)
#
# OSATS_CATEGORY_NAMES_LIST_TASK2  = [
#             'OSATS_RESPECT', 'OSATS_MOTION', 'OSATS_INSTRUMENT',
#             'OSATS_SUTURE', 'OSATS_FLOW', 'OSATS_KNOWLEDGE',
#             'OSATS_PERFORMANCE', 'OSATS_FINAL_QUALITY'
# ]
# from torchvision.transforms import Resize
#
# VIDEODATASET_MEAN_TENSOR = torch.tensor([0.5, 0.5, 0.5], device=device).view(1, 3, 1, 1)
# VIDEODATASET_STD_TENSOR = torch.tensor([0.5, 0.5, 0.5], device=device).view(1, 3, 1, 1)
#
# resnet_roi_transform = Compose([
#     Resize(RESNET_INPUT_SIZE, antialias=True),
#     Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Standard ImageNet normalization
# ])
#
# def denormalize_frame_for_detector(normalized_frame_chw_tensor_on_device):
#     # Input: (C,H,W) tensor, normalized by VideoDataset's mean/std
#     # Output: (C,H,W) tensor, range [0,1]
#     # Ajuste para operar em um tensor (C,H,W) sem unsqueeze/squeeze desnecessários se as constantes são (C,1,1)
#     mean = VIDEODATASET_MEAN_TENSOR.squeeze() # (C)
#     std = VIDEODATASET_STD_TENSOR.squeeze()   # (C)
#     if mean.ndim == 1: # Se for (C), reshape para (C,1,1) para broadcasting
#         mean = mean.view(-1, 1, 1)
#         std = std.view(-1, 1, 1)
#
#     frame_0_1 = normalized_frame_chw_tensor_on_device * std + mean
#     return frame_0_1.clamp_(0, 1)
#
#
# def get_best_roi_from_yolo_pred_and_crop(
#     yolo_pred_single_image, # Objeto results[i] do YOLO para uma imagem
#     original_frame_chw_on_device: torch.Tensor, # (C,H,W) de onde cortar, já no device
#     score_threshold: float = 0.25,
#     target_class_idx: int = None
# ) -> torch.Tensor:
#     """
#     Processa a previsão do YOLO para uma única imagem, seleciona a melhor RoI,
#     e a corta do frame original.
#     """
#     best_box_coords = None
#     max_score = score_threshold
#
#     if hasattr(yolo_pred_single_image, 'boxes') and yolo_pred_single_image.boxes is not None:
#         boxes_xyxy = yolo_pred_single_image.boxes.xyxy  # Tensor (N, 4) no device
#         confs = yolo_pred_single_image.boxes.conf    # Tensor (N)
#         clss = yolo_pred_single_image.boxes.cls      # Tensor (N)
#
#         for i in range(boxes_xyxy.shape[0]):
#             current_score = confs[i].item()
#             current_class_idx = int(clss[i].item())
#
#             if target_class_idx is not None and current_class_idx != target_class_idx:
#                 continue
#             if current_score > max_score:
#                 max_score = current_score
#                 box = boxes_xyxy[i]
#                 best_box_coords = (box[0].item(), box[1].item(), box[2].item(), box[3].item())
#
#     C, H, W = original_frame_chw_on_device.shape
#     if best_box_coords is not None:
#         xmin, ymin, xmax, ymax = best_box_coords
#         top, left, height, width = int(ymin), int(xmin), int(ymax - ymin), int(xmax - xmin)
#
#         # Ajustes para garantir que o corte seja válido
#         left = max(0, left)
#         top = max(0, top)
#         if left + width > W: width = W - left
#         if top + height > H: height = H - top
#
#         if height > 0 and width > 0:
#             try:
#                 return TF.crop(original_frame_chw_on_device, top, left, height, width)
#             except Exception as e:
#                 print(f"Error in TF.crop (box: {best_box_coords}, adj: t:{top} l:{left} h:{height} w:{width}): {e}")
#
#     # Fallback: região central
#     # print(f"Warning: No valid RoI for one sample. Using central region.")
#     ph, pw = H // 2, W // 2
#     pt, pl = (H - ph) // 2, (W - pw) // 2
#     if pt >= 0 and pl >= 0 and ph > 0 and pw > 0:
#         return TF.crop(original_frame_chw_on_device, pt, pl, ph, pw)
#     return torch.zeros((C, 32, 32), device=original_frame_chw_on_device.device, dtype=original_frame_chw_on_device.dtype)
#
# class Score_Classifier_Heads_Task2(nn.Module):
#     # Sua definição está boa, sem alterações necessárias aqui.
#     def __init__(self, num_osats_categories, num_scores_per_osats_cat, backbone_features):
#         super().__init__()
#         self.osats_heads = nn.ModuleList([
#             nn.Linear(backbone_features, num_scores_per_osats_cat)
#             for _ in range(num_osats_categories)
#         ])
#
#     def forward(self, backbone_features_batch):
#         head_outputs = [head(backbone_features_batch) for head in self.osats_heads]
#         stacked_outputs = torch.stack(head_outputs, dim=1)
#         return stacked_outputs.permute(0, 2, 1)
#
#
# class TwoStage_VideoScoringModel_YOLO_Task2(nn.Module):
#     def __init__(self, yolo_model_name='yolov8n.pt', detector_score_thresh=0.25,
#                  target_yolo_class_idx=None, frame_to_process_idx=None):
#         super().__init__()
#         self.detector_score_threshold = detector_score_thresh
#         self.target_yolo_class_idx = target_yolo_class_idx
#         self.frame_to_process_idx = frame_to_process_idx
#
#         print(f"Initializing YOLO detector ({yolo_model_name}) on device: {device}")
#         self.detector = YOLO(yolo_model_name)
#         self.detector.to(device)
#         self.detector.eval() # Detector sempre em eval
#         print("Freezing YOLO detector parameters.")
#         for param in self.detector.parameters(): # Congelar pesos do YOLO
#             param.requires_grad = False
#
#         print("Initializing ResNet50 backbone for RoI feature extraction...")
#         self.roi_feature_extractor = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
#         backbone_out_features = self.roi_feature_extractor.fc.in_features
#         self.roi_feature_extractor.fc = nn.Identity()
#         self.roi_feature_extractor.to(device)
#
#         print("Initializing Score Classifier heads for Task 2...")
#         self.score_classifier = Score_Classifier_Heads_Task2(
#             NUM_OSATS_CATEGORIES, NUM_SCORES_PER_CATEGORY, backbone_out_features
#         ).to(device)
#
#         self.roi_transform = resnet_roi_transform
#
#     def forward(self, video_clips_batch):
#         # video_clips_batch: (B, C, T, H_processed, W_processed) no device
#         batch_size, C, T, H_proc, W_proc = video_clips_batch.shape
#
#         # 1. Selecionar frames representativos para YOLO (em lote)
#         idx_to_pick = T // 2
#         if self.frame_to_process_idx is not None:
#             idx_to_pick = min(self.frame_to_process_idx, T - 1)
#         # representative_frames_ds_norm: (B, C, H_proc, W_proc)
#         representative_frames_ds_norm = video_clips_batch[:, :, idx_to_pick, :, :]
#
#         # 2. Denormalizar frames para o detector (em lote, se possível, ou em loop)
#         # Esta denormalização precisa ser cuidadosa. Se VIDEODATASET_MEAN/STD_TENSOR são para (B,C,H,W)
#         # ou (C,H,W) ou (C), a operação muda.
#         # Assumindo que denormalize_frame_for_detector lida com (C,H,W) e retorna (C,H,W)
#         frames_for_detector_0_1_list = [
#             denormalize_frame_for_detector(representative_frames_ds_norm[i]) for i in range(batch_size)
#         ]
#         frames_for_detector_0_1_batch = torch.stack(frames_for_detector_0_1_list) # (B, C, H_proc, W_proc)
#
#         # 3. Obter Detecções YOLO (em lote)
#         # O detector já está em eval() e no device correto
#         yolo_preds_list = self.detector(frames_for_detector_0_1_batch, verbose=False) # Lista de objetos Results
#
#         # 4. Processar RoIs para cada amostra no lote
#         processed_rois_for_batch = []
#         for i in range(batch_size):
#             yolo_pred_single = yolo_preds_list[i]
#             # Usar o frame denormalizado [0,1] de onde cortar
#             original_frame_0_1_single = frames_for_detector_0_1_batch[i]
#
#             roi_0_1_tensor = get_best_roi_from_yolo_pred_and_crop(
#                 yolo_pred_single,
#                 original_frame_0_1_single,
#                 self.detector_score_threshold,
#                 self.target_yolo_class_idx
#             ) # Retorna no device
#
#             # Transformar RoI para ResNet
#             processed_roi = self.roi_transform(roi_0_1_tensor) # Espera (C,H,W) no device
#             processed_rois_for_batch.append(processed_roi)
#
#         # Empilhar RoIs
#         if not processed_rois_for_batch: # Pouco provável se o fallback de RoI funcionar
#             print("CRITICAL ERROR: No RoIs were processed for the batch, even with fallback. Check get_best_roi_from_yolo_pred_and_crop.")
#             # Retornar zeros pode quebrar o treinamento se as dimensões não baterem com o esperado pela loss
#             # É melhor garantir que processed_rois_for_batch nunca esteja vazio
#             # Esta parte do código não deveria ser alcançada se o fallback de RoI funcionar.
#             # Se chegar aqui, há um problema maior.
#             # Por segurança, crie um lote de RoIs de placeholder se estiver vazio.
#             placeholder_roi = self.roi_transform(torch.zeros((C, H_proc, W_proc), device=device))
#             final_rois_tensor_batch = torch.stack([placeholder_roi] * batch_size)
#         else:
#             final_rois_tensor_batch = torch.stack(processed_rois_for_batch)
#
#         # 5. Extrair Features e Classificar
#         # O estado .train() / .eval() dos submódulos é gerenciado pelo
#         # model.train() / model.eval() chamado no loop de treinamento externo.
#         # O detector YOLO permanece em eval() como definido no __init__.
#
#         roi_features = self.roi_feature_extractor(final_rois_tensor_batch)
#         scores = self.score_classifier(roi_features)
#         return scores
#

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim # Ensure optim is imported

# Assume device is defined globally or passed appropriately
# Assume NUM_SCORES_PER_CATEGORY = 5 is globally available or passed

def train_model(model, train_loader, val_loader, num_epochs=5, learning_rate=0.001, current_task_type="task2"):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

    # This should be accessible if defined in the same scope or globally
    # For this example, let's explicitly state it for clarity in this function,
    # but ideally, it's passed or accessed from a config.
    # NUM_SCORES_PER_CATEGORY_CONST = 5 # As per your global definition

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct_osats_scores = 0
        train_total_osats_scores = 0

        print(f"Starting Epoch: {epoch+1}/{num_epochs}")
        for videos, labels in train_loader:
            if videos.numel() == 0 or labels.numel() == 0 : continue
            videos = videos.to(device) # (B, C, T, H, W)
            original_labels = labels.to(device, dtype=torch.long) # For Task 2: (B, NUM_OSATS_CATEGORIES)

            # --- IMPORTANT MODIFICATION: Convert labels to 0-indexed ---
            # Assuming original labels are 1-indexed (e.g., 1 to NUM_SCORES_PER_CATEGORY)
            # We need them to be 0-indexed (e.g., 0 to NUM_SCORES_PER_CATEGORY - 1)
            # A quick check: if labels.min() < 0 or labels.max() >= NUM_SCORES_PER_CATEGORY (using 0-indexed logic for max)
            # For 1-indexed, they would be 1 to NUM_SCORES_PER_CATEGORY
            # So, if labels.min() < 1 or labels.max() > NUM_SCORES_PER_CATEGORY, there's an issue.
            # The error "Target 5 is out of bounds" for NUM_SCORES_PER_CATEGORY=5 confirms 1-indexing is likely.
            labels_0_indexed = original_labels - 1
            # ------------------------------------------------------------

            optimizer.zero_grad()
            outputs = model(videos) # For Task 2: (B, NUM_SCORES_PER_CATEGORY, NUM_OSATS_CATEGORIES)

            # Task 2 specific loss and accuracy, using 0-indexed labels
            loss = criterion(outputs, labels_0_indexed)
            _, predicted_scores = torch.max(outputs, 1) # Max over NUM_SCORES_PER_CATEGORY dim -> (B, NUM_OSATS_CATEGORIES)
                                                        # predicted_scores are already 0-indexed (0 to C-1)

            train_total_osats_scores += labels_0_indexed.numel() # Total individual OSATS scores in batch
            train_correct_osats_scores += (predicted_scores == labels_0_indexed).sum().item()

            loss.backward()
            optimizer.step()
            train_loss += loss.item() * videos.size(0)


        avg_train_loss = train_loss / len(train_loader.dataset) if len(train_loader.dataset) > 0 else 0
        train_accuracy = train_correct_osats_scores / train_total_osats_scores if train_total_osats_scores > 0 else 0

        model.eval()
        val_loss = 0.0
        val_correct_osats_scores = 0
        val_total_osats_scores = 0
        with torch.no_grad():
            for videos, labels in val_loader:
                if videos.numel() == 0 or labels.numel() == 0 : continue
                videos = videos.to(device)
                original_labels = labels.to(device, dtype=torch.long)

                # --- IMPORTANT MODIFICATION: Convert labels to 0-indexed ---
                labels_0_indexed = original_labels - 1
                # ------------------------------------------------------------

                outputs = model(videos)

                loss = criterion(outputs, labels_0_indexed)
                _, predicted_scores = torch.max(outputs, 1) # predicted_scores are 0-indexed
                val_total_osats_scores += labels_0_indexed.numel()
                val_correct_osats_scores += (predicted_scores == labels_0_indexed).sum().item()
                val_loss += loss.item() * videos.size(0)

        avg_val_loss = val_loss / len(val_loader.dataset) if len(val_loader.dataset) > 0 else 0
        val_accuracy = val_correct_osats_scores / val_total_osats_scores if val_total_osats_scores > 0 else 0

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f}, "
              f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")
    return model

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim # Make sure optim is imported
from torchvision.transforms import Compose, Resize, Normalize
import torchvision.transforms.functional as TF

# Conditional YOLO import for environments where it might not be installed
try:
    from ultralytics import YOLO
except ImportError:
    print("Warning: ultralytics.YOLO not found. Using a MockYOLO class. Training will not be functional with real YOLO.")
    class MockYOLOResults: # Minimal mock for YOLO results
        def __init__(self):
            self.boxes = None # In a real scenario, this would have xyxy, conf, cls attributes
    class MockYOLO(nn.Module):
        def __init__(self, model_name='yolov8n.pt'):
            super().__init__()
            self.model_name = model_name
            self.dummy_param = nn.Parameter(torch.empty(0)) # To have parameters
        def forward(self, imgs, verbose=False):
            if isinstance(imgs, torch.Tensor): batch_size = imgs.shape[0]
            elif isinstance(imgs, list): batch_size = len(imgs)
            else: batch_size = 1
            return [MockYOLOResults() for _ in range(batch_size)] # List of result objects
        # train and eval are inherited from nn.Module and are fine for a mock
    YOLO = MockYOLO


# --- Your existing constants and helper functions ---
# (Assuming device, NUM_OSATS_CATEGORIES, etc. are defined as in your snippet)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_OSATS_CATEGORIES = 8
NUM_SCORES_PER_CATEGORY = 5
RESNET_INPUT_SIZE = (224, 224)

# Make sure VIDEODATASET_MEAN_TENSOR and VIDEODATASET_STD_TENSOR are correctly defined
# For denormalize_frame_for_detector to work with (C,H,W) frames, mean/std should broadcast
# e.g. shape (3) or (3,1,1) if VIDEODATASET_MEAN_TENSOR is on CPU, then move to device.
# Or if already on device, ensure correct shape.
# Assuming they are (C,) and will be reshaped in denormalize_frame_for_detector
VIDEODATASET_MEAN_TENSOR = torch.tensor([0.5, 0.5, 0.5], device=device) #.view(1, 3, 1, 1)
VIDEODATASET_STD_TENSOR = torch.tensor([0.5, 0.5, 0.5], device=device) #.view(1, 3, 1, 1)

resnet_roi_transform = Compose([
    Resize(RESNET_INPUT_SIZE, antialias=True),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def denormalize_frame_for_detector(normalized_frame_chw_tensor_on_device):
    mean = VIDEODATASET_MEAN_TENSOR # Should be on the same device
    std = VIDEODATASET_STD_TENSOR   # Should be on the same device
    if mean.ndim == 1:
        mean = mean.view(-1, 1, 1)
        std = std.view(-1, 1, 1)
    # Ensure mean and std are on the same device as the input tensor
    frame_0_1 = normalized_frame_chw_tensor_on_device * std.to(normalized_frame_chw_tensor_on_device.device) + \
                mean.to(normalized_frame_chw_tensor_on_device.device)
    return frame_0_1.clamp_(0, 1)


def get_best_roi_from_yolo_pred_and_crop(
    yolo_pred_single_image,
    original_frame_chw_on_device: torch.Tensor,
    score_threshold: float = 0.75,
    target_class_idx: int = None
) -> torch.Tensor:
    best_box_coords = None
    max_score = score_threshold

    if hasattr(yolo_pred_single_image, 'boxes') and yolo_pred_single_image.boxes is not None and \
       hasattr(yolo_pred_single_image.boxes, 'xyxy') and yolo_pred_single_image.boxes.xyxy is not None: # Check for actual data
        boxes_xyxy = yolo_pred_single_image.boxes.xyxy
        confs = yolo_pred_single_image.boxes.conf
        clss = yolo_pred_single_image.boxes.cls

        for i in range(boxes_xyxy.shape[0]):
            current_score = confs[i].item()
            current_class_idx = int(clss[i].item())

            if target_class_idx is not None and current_class_idx != target_class_idx:
                continue
            if current_score > max_score:
                max_score = current_score
                box = boxes_xyxy[i]
                best_box_coords = (box[0].item(), box[1].item(), box[2].item(), box[3].item())

    C, H, W = original_frame_chw_on_device.shape
    if best_box_coords is not None:
        xmin, ymin, xmax, ymax = best_box_coords
        # Ensure coordinates are within image bounds and form a valid box
        xmin = max(0, min(xmin, W -1))
        ymin = max(0, min(ymin, H -1))
        xmax = max(xmin, min(xmax, W -1)) # ensure xmax >= xmin
        ymax = max(ymin, min(ymax, H -1)) # ensure ymax >= ymin

        top, left = int(ymin), int(xmin)
        height, width = int(ymax - ymin), int(xmax - xmin)

        if height > 0 and width > 0:
            try:
                return TF.crop(original_frame_chw_on_device, top, left, height, width)
            except Exception as e:
                print(f"Error in TF.crop (box: {best_box_coords}, adj: t:{top} l:{left} h:{height} w:{width}): {e}")
        else:
            print(f"Warning: Invalid RoI dimensions after clamping (h:{height}, w:{width}). Using fallback.")


    # Fallback: região central
    ph, pw = H // 2, W // 2
    pt, pl = (H - ph) // 2, (W - pw) // 2
    if pt >= 0 and pl >= 0 and ph > 0 and pw > 0: # Ensure fallback is valid
        return TF.crop(original_frame_chw_on_device, pt, pl, ph, pw)
    # Ultimate fallback: small zero tensor if central crop also fails (e.g., H or W too small)
    return torch.zeros((C, min(H,32), min(W,32)), device=original_frame_chw_on_device.device, dtype=original_frame_chw_on_device.dtype)


class Score_Classifier_Heads_Task2(nn.Module):
    def __init__(self, num_osats_categories, num_scores_per_osats_cat, backbone_features):
        super().__init__()
        self.osats_heads = nn.ModuleList([
            nn.Linear(backbone_features, num_scores_per_osats_cat)
            for _ in range(num_osats_categories)
        ])
    def forward(self, backbone_features_batch):
        head_outputs = [head(backbone_features_batch) for head in self.osats_heads]
        stacked_outputs = torch.stack(head_outputs, dim=1) # (B, NUM_OSATS_CATEGORIES, NUM_SCORES_PER_CATEGORY)
        return stacked_outputs.permute(0, 2, 1) # (B, NUM_SCORES_PER_CATEGORY, NUM_OSATS_CATEGORIES)


class TwoStage_VideoScoringModel_YOLO_Task2(nn.Module):
    def __init__(self, yolo_model_name='yolov8n.pt', detector_score_thresh=0.25,
                 target_yolo_class_idx=None, frame_to_process_idx=None):
        super().__init__()
        self.detector_score_threshold = detector_score_thresh
        self.target_yolo_class_idx = target_yolo_class_idx
        self.frame_to_process_idx = frame_to_process_idx

        print(f"Initializing YOLO detector ({yolo_model_name}) on device: {device}")
        self.detector = YOLO(yolo_model_name)
        self.detector.to(device)
        self.detector.eval() # Set to eval mode initially
        print("Freezing YOLO detector parameters.")
        for param in self.detector.parameters():
            param.requires_grad = False

        print("Initializing ResNet50 backbone for RoI feature extraction...")
        self.roi_feature_extractor = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        backbone_out_features = self.roi_feature_extractor.fc.in_features
        self.roi_feature_extractor.fc = nn.Identity()
        self.roi_feature_extractor.to(device) # Ensure ResNet is also on the device

        print("Initializing Score Classifier heads for Task 2...")
        self.score_classifier = Score_Classifier_Heads_Task2(
            NUM_OSATS_CATEGORIES, NUM_SCORES_PER_CATEGORY, backbone_out_features
        ).to(device) # Ensure classifier is also on the device

        self.roi_transform = resnet_roi_transform

    # --- FIX IS HERE ---
    def train(self, mode: bool = True):
        """
        Override the default train method to:
        1. Set the training mode for this module itself.
        2. Set the training mode for trainable submodules (ResNet, classifier).
        3. Explicitly keep the YOLO detector in eval() mode.
        """
        self.training = mode # Set mode for this nn.Module
        # Set mode for children modules that should be affected by train/eval
        self.roi_feature_extractor.train(mode)
        self.score_classifier.train(mode)
        # Keep the YOLO detector in eval mode always
        if self.detector is not None:
            self.detector.eval()
        return self

    def forward(self, video_clips_batch):
        # video_clips_batch: (B, C, T, H_processed, W_processed) on device
        batch_size, C, T, H_proc, W_proc = video_clips_batch.shape

        idx_to_pick = T // 2
        if self.frame_to_process_idx is not None:
            idx_to_pick = min(self.frame_to_process_idx, T - 1)
        representative_frames_ds_norm = video_clips_batch[:, :, idx_to_pick, :, :]

        frames_for_detector_0_1_list = [
            denormalize_frame_for_detector(representative_frames_ds_norm[i]) for i in range(batch_size)
        ]
        frames_for_detector_0_1_batch = torch.stack(frames_for_detector_0_1_list)

        with torch.no_grad(): # Ensure YOLO prediction runs without gradient tracking
            yolo_preds_list = self.detector(frames_for_detector_0_1_batch, verbose=False)

        processed_rois_for_batch = []
        for i in range(batch_size):
            yolo_pred_single = yolo_preds_list[i]
            original_frame_0_1_single = frames_for_detector_0_1_batch[i]
            roi_0_1_tensor = get_best_roi_from_yolo_pred_and_crop(
                yolo_pred_single,
                original_frame_0_1_single,
                self.detector_score_threshold,
                self.target_yolo_class_idx
            )
            processed_roi = self.roi_transform(roi_0_1_tensor)
            processed_rois_for_batch.append(processed_roi)

        if not processed_rois_for_batch: # Should be rare with fallbacks
            print("CRITICAL FALLBACK: No RoIs processed. Creating placeholder RoIs.")
            # Create a batch of placeholder RoIs, transformed
            placeholder_frame = torch.zeros((C, H_proc, W_proc), device=device, dtype=video_clips_batch.dtype)
            placeholder_roi_transformed = self.roi_transform(placeholder_frame)
            final_rois_tensor_batch = torch.stack([placeholder_roi_transformed] * batch_size)
        else:
            final_rois_tensor_batch = torch.stack(processed_rois_for_batch)

        # roi_feature_extractor and score_classifier will be in train or eval mode
        # based on the outer model's mode set by the overridden train() method.
        roi_features = self.roi_feature_extractor(final_rois_tensor_batch)
        scores = self.score_classifier(roi_features)
        return scores

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models



if train_dl and val_dl and len(train_dl.dataset) > 0 and len(val_dl.dataset) > 0:
    video_scoring_model_task2 = TwoStage_VideoScoringModel_YOLO_Task2(
        yolo_model_name='yolov8n.pt',
        detector_score_thresh=1,
        target_yolo_class_idx=None,
        frame_to_process_idx=segments // 2
    ).to(device)


    total_params = sum(p.numel() for p in video_scoring_model_task2.parameters())
    trainable_params = sum(p.numel() for p in video_scoring_model_task2.parameters() if p.requires_grad)
    print(f"Total model parameters: {total_params:,}")
    print(f"Trainable model parameters: {trainable_params:,}")

    trained_model_task2 = train_model(
        video_scoring_model_task2,
        train_dl,
        val_dl,
        num_epochs=1,
        learning_rate=1e-4
    )

    torch.save(trained_model_task2.state_dict(), "YOLO_RESNET50_V1_state_dict.pth")
    print("Model state_dict saved to YOLO_RESNET50_V1_state_dict.pth")

else:
    print(f"Could not prepare dataloaders for {task_to_run} or datasets are empty. Skipping model instantiation and training.")
    if train_dl: print(f"Train Dataloader dataset size: {len(train_dl.dataset)}")
    if val_dl: print(f"Validation Dataloader dataset size: {len(val_dl.dataset)}")

Initializing YOLO detector (yolov8n.pt) on device: cpu
Freezing YOLO detector parameters.
Initializing ResNet50 backbone for RoI feature extraction...
Initializing Score Classifier heads for Task 2...
Total model parameters: 26,747,192
Trainable model parameters: 23,589,992
Starting Epoch: 1/1
Epoch 1/1, Train Loss: 1.5055, Train Acc: 0.3571, Val Loss: 1.4909, Val Acc: 0.3077
Model state_dict saved to YOLO_RESNET50_V1_state_dict.pth


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import mean_absolute_error, f1_score, accuracy_score, classification_report, precision_recall_fscore_support
import traceback

# ... (rest of your imports and model/constants definitions) ...

def evaluate_model_task2(model, test_loader, device,
                         num_osats_categories,
                         num_scores_per_category,
                         osats_category_names=None):
    model = model.to(device)
    model.eval()
    criterion = nn.CrossEntropyLoss(reduction='mean')
    accumulated_loss_x_batch_size = 0.0
    total_samples_processed = 0
    all_predictions_list = []
    all_true_labels_list = []

    with torch.no_grad():
        for videos, original_labels_from_loader in test_loader:
            if videos.numel() == 0 or original_labels_from_loader.numel() == 0:
                print("Aviso: Batch vazio encontrado e pulado.")
                continue
            current_batch_size = videos.size(0)
            videos = videos.to(device)
            original_labels_1_indexed = original_labels_from_loader.to(device, dtype=torch.long)
            labels_0_indexed = original_labels_1_indexed - 1
            outputs = model(videos)
            loss = criterion(outputs, labels_0_indexed)
            accumulated_loss_x_batch_size += loss.item() * current_batch_size
            total_samples_processed += current_batch_size
            _, predicted_scores_0_indexed = torch.max(outputs, dim=1)
            all_predictions_list.append(predicted_scores_0_indexed.cpu().numpy())
            all_true_labels_list.append(labels_0_indexed.cpu().numpy())

    if not all_true_labels_list or total_samples_processed == 0:
        print("Nenhum dado processado no test_loader.")
        return 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, {}

    predictions_np = np.concatenate(all_predictions_list, axis=0)
    true_labels_np = np.concatenate(all_true_labels_list, axis=0)
    avg_test_loss = accumulated_loss_x_batch_size / total_samples_processed if total_samples_processed > 0 else 0.0
    test_mae_overall = mean_absolute_error(true_labels_np, predictions_np)

    per_category_f1_scores_macro = []
    per_category_f1_scores_weighted = []
    supports_for_weighted_f1 = []

    if num_osats_categories > 0:
        for i in range(num_osats_categories):
            true_cat_labels = true_labels_np[:, i]
            pred_cat_labels = predictions_np[:, i]
            f1_cat_macro = f1_score(true_cat_labels, pred_cat_labels, average='macro', zero_division=0)
            f1_cat_weighted = f1_score(true_cat_labels, pred_cat_labels, average='weighted', zero_division=0)
            per_category_f1_scores_macro.append(f1_cat_macro)
            per_category_f1_scores_weighted.append(f1_cat_weighted)
            supports_for_weighted_f1.append(len(true_cat_labels))

    test_f1_macro_overall = np.mean(per_category_f1_scores_macro) if per_category_f1_scores_macro else 0.0
    if per_category_f1_scores_weighted and sum(supports_for_weighted_f1) > 0:
        test_f1_weighted_overall = np.average(per_category_f1_scores_weighted, weights=supports_for_weighted_f1)
    else:
        test_f1_weighted_overall = 0.0

    # --- REVISED F1 SAMPLES ---
    sample_f1_scores = []
    if true_labels_np.shape[0] > 0 and num_osats_categories > 0: # Ensure there are samples and categories
        for i in range(true_labels_np.shape[0]): # Iterate over each sample
            true_sample_labels = true_labels_np[i, :] # (num_osats_categories,)
            pred_sample_labels = predictions_np[i, :] # (num_osats_categories,)

            # For this sample, how many categories were correctly predicted?
            tp_sample = np.sum(true_sample_labels == pred_sample_labels)

            # In multiclass-multioutput, for a single sample, each output slot is an "item".
            # Precision = TP / (TP + FP)
            # Recall = TP / (TP + FN)
            # For a single sample, TP+FP = num_osats_categories (total predicted items for the sample)
            # For a single sample, TP+FN = num_osats_categories (total true items for the sample)

            precision_sample = tp_sample / num_osats_categories if num_osats_categories > 0 else 0.0
            recall_sample = tp_sample / num_osats_categories if num_osats_categories > 0 else 0.0

            if precision_sample + recall_sample == 0:
                f1_s = 0.0
            else:
                f1_s = 2 * (precision_sample * recall_sample) / (precision_sample + recall_sample)
            sample_f1_scores.append(f1_s)

        test_f1_samples = np.mean(sample_f1_scores) if sample_f1_scores else 0.0
    else:
        test_f1_samples = 0.0
    # --- END OF REVISED F1 SAMPLES ---

    per_category_accuracy_list = []
    avg_accuracy_across_categories = 0.0
    if num_osats_categories > 0:
        for i in range(num_osats_categories):
            acc = accuracy_score(true_labels_np[:, i], predictions_np[:, i])
            per_category_accuracy_list.append(acc)
        if per_category_accuracy_list:
            avg_accuracy_across_categories = np.mean(per_category_accuracy_list)

    exact_match_ratio = 0.0
    if true_labels_np.shape[0] > 0: # Check if there are any samples
        # Compare entire rows. np.all(axis=1) checks if all elements in a row are True.
        # (true_labels_np == predictions_np) gives a boolean array of same shape.
        # np.all(..., axis=1) reduces it to (num_samples,), True if all outputs matched for that sample.
        # np.sum(...) counts how many samples had all outputs matching.
        # Then divide by the total number of samples.
        num_exact_matches = np.sum(np.all(true_labels_np == predictions_np, axis=1))
        exact_match_ratio = num_exact_matches / true_labels_np.shape[0]

    print(f"\n--- Métricas de Avaliação no Conjunto de Teste (Task 2) ---")
    print(f"Loss Médio por Predição Individual (CrossEntropy): {avg_test_loss:.4f}")
    print(f"Mean Absolute Error (MAE Médio por Categoria, 0-indexed): {test_mae_overall:.4f}")
    print(f"F1 Score (Macro Overall - Média dos F1-macro por Categoria): {test_f1_macro_overall:.4f}")
    print(f"F1 Score (Weighted Overall - Média Ponderada dos F1-weighted por Categoria): {test_f1_weighted_overall:.4f}")
    print(f"F1 Score (Samples - Média das F1 por Amostra): {test_f1_samples:.4f}")
    print(f"Acurácia (Média por Categoria OSATS): {avg_accuracy_across_categories:.4f}")
    print(f"Acurácia (Exact Match Ratio / Correspondência Exata do Subconjunto): {exact_match_ratio:.4f}")

    print("\n--- Relatório de Classificação Detalhado por Categoria OSATS ---")
    # ... (rest of your classification report logic, which is fine) ...
    detailed_reports = {}
    if osats_category_names is None or len(osats_category_names) != num_osats_categories:
        if num_osats_categories > 0:
            print(f"Aviso: Nomes das categorias OSATS não fornecidos ou com tamanho incorreto. Usando nomes genéricos.")
        osats_category_names_internal = [f"Categoria_{i+1}" for i in range(num_osats_categories)]
    else:
        osats_category_names_internal = osats_category_names

    report_class_indices = list(range(num_scores_per_category))
    target_names_for_scores = [f"Score_0idx {j}" for j in report_class_indices]

    if num_osats_categories > 0:
        for i in range(num_osats_categories):
            cat_name = osats_category_names_internal[i]
            print(f"\n-- {cat_name} --")
            true_cat_labels = true_labels_np[:, i]
            pred_cat_labels = predictions_np[:, i]
            try:
                report_str = classification_report(
                    true_cat_labels,
                    pred_cat_labels,
                    labels=report_class_indices,
                    target_names=target_names_for_scores,
                    zero_division=0,
                    digits=3
                )
                print(report_str)
                detailed_reports[cat_name] = report_str
            except ValueError as e:
                print(f"Não foi possível gerar o relatório de classificação para {cat_name}: {e}")
                if true_cat_labels is not None: print(f"Labels verdadeiros únicos para {cat_name}: {np.unique(true_cat_labels)}")
                if pred_cat_labels is not None: print(f"Labels previstos únicos para {cat_name}: {np.unique(pred_cat_labels)}")
                detailed_reports[cat_name] = f"Erro ao gerar relatório: {e}"
            except IndexError as e:
                print(f"Erro de índice ao acessar categoria {i} para {cat_name}: {e}")
                detailed_reports[cat_name] = f"Erro de índice: {e}"
    elif total_samples_processed > 0:
        print("Nenhuma categoria OSATS definida (num_osats_categories = 0), relatório detalhado por categoria não aplicável.")


    return (avg_test_loss, test_mae_overall, test_f1_macro_overall, test_f1_weighted_overall,
            test_f1_samples, avg_accuracy_across_categories, exact_match_ratio, detailed_reports)

model_path = "YOLO_RESNET50_V1_state_dict.pth"

model_task2_eval = TwoStage_VideoScoringModel_YOLO_Task2(
        yolo_model_name='yolov8n.pt',
        detector_score_thresh=1,
        target_yolo_class_idx=None,
        frame_to_process_idx=segments // 2
    ).to(device)

print(f"Verificando model_task2_eval: {type(model_task2_eval)}")


try:
    # 2. Carregue o dicionário de pesos do arquivo
    print(f"Tentando carregar o arquivo de pesos de: {model_path}")
    loaded_state_dict = torch.load(model_path, map_location=device)
    print("Arquivo de pesos carregado na memória com sucesso.")

    # 3. Use o MÉTODO .load_state_dict() DO SEU OBJETO model_task2_eval
    #    Passe strict=False para ignorar as chaves não correspondentes do detector YOLO.
    print("Tentando carregar os pesos no modelo (strict=False)...")
    missing_keys, unexpected_keys = model_task2_eval.load_state_dict(loaded_state_dict, strict=False)

    print(f"Pesos carregados no modelo a partir de {model_path}")

    # Opcional: Imprimir informações sobre chaves não correspondentes para verificação
    # if missing_keys:
    #     print("\nAviso: As seguintes chaves estavam faltando no state_dict carregado e não foram carregadas (o modelo usou seus pesos iniciais para estas):")
    #     for key in missing_keys:
    #         print(f"  - {key}")
    # if unexpected_keys:
    #     print("\nAviso: As seguintes chaves estavam no state_dict carregado mas não na arquitetura do modelo e foram ignoradas:")
    #     for key in unexpected_keys:
    #         print(f"  - {key}")

    # if not missing_keys and not unexpected_keys:
    #     print("Todas as chaves corresponderam perfeitamente.")
    # elif all(k.startswith("detector.") for k in missing_keys + unexpected_keys):
    #     print("\nTodas as não correspondências estavam dentro do módulo 'detector.'. Isto é esperado se o detector estava congelado ou sua estrutura mudou (ex: devido à fusão de camadas).")
    #     print("Os pesos para as partes treináveis (ResNet, classificadores) devem ter sido carregados corretamente.")
    # else:
    #     print("\nAviso: Houve não correspondências além do módulo 'detector.'. Por favor, revise cuidadosamente se outras partes deveriam ter sido treinadas e carregadas.")

    # 4. Prossiga com a avaliação
    # Certifique-se de que test_dl está definido corretamente e não está vazio
    if test_dl and len(test_dl.dataset) > 0:
        print("\nProsseguindo com a avaliação...")
        results = evaluate_model_task2(
            model_task2_eval,
            test_dl,
            device,
            num_osats_categories=8,
            num_scores_per_category=5,
            osats_category_names=['OSATS_RESPECT', 'OSATS_MOTION', 'OSATS_INSTRUMENT',
                                  'OSATS_SUTURE', 'OSATS_FLOW', 'OSATS_KNOWLEDGE',
                                  'OSATS_PERFORMANCE', 'OSATS_FINAL_QUALITY']
        )
        print("\nTupla de Resultados da Avaliação:", results)
    else:
        print("\nPulando a avaliação porque test_dl não é válido ou está vazio.")
        if test_dl:
            print(f"Tamanho do dataset do Test Dataloader: {len(test_dl.dataset)}")
        else:
            print("Test Dataloader (test_dl) é None.")

except FileNotFoundError:
    print(f"Erro Crítico: Arquivo de modelo não encontrado em {model_path}. Não é possível carregar os pesos.")
except AttributeError as ae:
    print(f"Erro de Atributo: {ae}")
    print("Isso pode significar que 'model_task2_eval' não é um objeto de modelo PyTorch válido ou é 'None'.")
    print("Verifique a instanciação do seu modelo.")
except Exception as e:
    print(f"Ocorreu um erro durante o carregamento do state_dict ou avaliação: {e}")
    import traceback
    traceback.print_exc()

Initializing YOLO detector (yolov8n.pt) on device: cpu
Freezing YOLO detector parameters.
Initializing ResNet50 backbone for RoI feature extraction...
Initializing Score Classifier heads for Task 2...
Verificando model_task2_eval: <class '__main__.TwoStage_VideoScoringModel_YOLO_Task2'>
Tentando carregar o arquivo de pesos de: YOLO_RESNET50_V1_state_dict.pth
Arquivo de pesos carregado na memória com sucesso.
Tentando carregar os pesos no modelo (strict=False)...
Pesos carregados no modelo a partir de YOLO_RESNET50_V1_state_dict.pth

Prosseguindo com a avaliação...

--- Métricas de Avaliação no Conjunto de Teste (Task 2) ---
Loss Médio por Predição Individual (CrossEntropy): 1.5013
Mean Absolute Error (MAE Médio por Categoria, 0-indexed): 1.1875
F1 Score (Macro Overall - Média dos F1-macro por Categoria): 0.1671
F1 Score (Weighted Overall - Média Ponderada dos F1-weighted por Categoria): 0.2181
F1 Score (Samples - Média das F1 por Amostra): 0.3214
Acurácia (Média por Categoria OSATS): 0

# Para baixo é lixo ( mas nao para apagar )